<a href="https://colab.research.google.com/github/knw5289/Parent-Paper-1/blob/main/Modified_Implmentation_of_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [ ]:
#Installations
!pip install dash
!pip install pymysql

## Dataset for Dashboard

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import random
import nltk
from nltk.corpus import stopwords
import warnings

warnings.filterwarnings('ignore')

list_of_us_codes = ['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI',
                    'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP',
                    'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
                    'SC', 'SD', 'TN', 'TX', 'UM', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']

nltk.download('stopwords')
stopwords = stopwords.words('english')
stopwords.append('')

df = pd.DataFrame(columns = ['posted_date', 'state', 'term', 'count'])
resumesurl = 'https://raw.githubusercontent.com/knw5289/Parent-Paper-1/main/UpdatedResumeDataSet.csv'
resumes = pd.read_csv(resumesurl)
lexiconurl = 'https://raw.githubusercontent.com/knw5289/Parent-Paper-1/main/Job%20Description%20Dataset%20-%20Sheet1.csv'
lexicon = pd.read_csv(lexiconurl)
resumes10 = resumes.head(10) #For testing purposes
used_category = []
skillsdict = {}
for index, info in resumes.iterrows():
  words = info[1].split()
  words = list(map(str.lower,words))
  skills = lexicon.loc[lexicon['Category'] == info[0]]
  skillslist = skills['Description'].tolist()
  skillslist = list(map(str.lower,skillslist))
  skillwords = []
  for word in skillslist:
    wordssplit = word.split()
    for word in wordssplit:
      skillwords.append(word)
  cleanedskillwords = []
  for word in skillwords:
    cleanedskillwords.append(''.join(letter for letter in word if letter.isalnum()))
  for word in cleanedskillwords:
    if word in stopwords:
      cleanedskillwords.remove(word)
  skillsdict[info[0]] = cleanedskillwords
  if info[0] not in used_category:
    used_category.append(info[0])
    for word in cleanedskillwords:
      if word not in list(df.columns):
        df.insert(4, word, [0]*index, False)
  df.loc[len(df)] = 0
  used_category.append(info[0])
  start_date = datetime(2020, 4, 20) #Randomly generated dates
  end_date = datetime(2020, 10, 30)
  num_days = (end_date - start_date).days
  rand_days = random.randint(1, num_days)
  random_date = start_date + timedelta(days=rand_days)
  df.at[index, 'posted_date'] = random_date
  df['posted_date'] = pd.to_datetime(df['posted_date']).dt.date
  df['posted_date'] = df['posted_date'].astype(str)
  df['posted_date'] = df['posted_date'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d'))
  df.at[index, 'state'] = random.choice(list_of_us_codes) #Randomly generated states
  df.at[index, 'term'] = info[0]
  df.at[index, 'count'] = 1
  cleanedwords = []
  for word in words:
    cleanedwords.append(''.join(letter for letter in word if letter.isalnum()))
  for word in cleanedwords:
    if word in cleanedskillwords and df.at[index, word] == 0:
      df.at[index, word] += 1

df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,posted_date,state,term,count,exploratory,system,hp,functional,jira,assurance,...,natural,deep,python,science,data,language,programming,r,learning,machine
0,2020-05-13,PA,Data Science,1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,1,1
1,2020-06-22,VI,Data Science,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,1
2,2020-07-16,IL,Data Science,1,0,0,0,0,0,0,...,0,1,1,1,1,0,1,0,1,1
3,2020-09-11,HI,Data Science,1,0,0,0,0,0,0,...,0,1,1,1,1,0,1,1,1,1
4,2020-09-11,NV,Data Science,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,2020-05-11,KS,Testing,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
958,2020-05-07,AZ,Testing,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
959,2020-08-04,MP,Testing,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
960,2020-08-15,WA,Testing,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Dashboard

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan  9 21:43:28 2021

@author: mojtaba heidarysafa
"""


#from app_sub import top_skill_list,us_state_abbrev,list_of_us_codes, modify_start_end_date
import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output, State
import plotly.express as px
from datetime import datetime as dt


#import pandas as pd
import time
import pymysql #version 0.9.2

from pandas.tseries.offsets import DateOffset

'''
list_of_us_codes = ['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI',
                    'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP',
                    'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
                    'SC', 'SD', 'TN', 'TX', 'UM', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']
'''

us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}



###### count plot prep weeks
def modify_start_end_date(date, forward=True):
    '''
    :param date: datetime for the date to modify
    :param forward: a boolean that specified if the date is moving forward
    to the beginning next week or back to the end of previous week
    :return: modified datetime
    '''
    if forward:
        if date.weekday() != 0:
            offset = 7 - date.weekday()
            start_date = date + DateOffset(offset)
            return start_date
        else:
            return date
    else:
        # modify the end date of dataframe to the end of last week
        if date.weekday() != 0:
            offset = date.weekday()
            end_date = date - DateOffset(offset + 1)
            return end_date
        else:
            return date

'''
###### in case of connecting to
host = "your_database_on_amazon.....rds.amazonaws.com"

port =int(3306)
dbname="job_posts"
user="yourusername"
password="yourpassword"
##### create connection with parameters you need
conn = pymysql.connect(host, user=user, port=port, passwd=password, db=dbname)
start ="""2020-04-10"""
end = """2020-05-10"""
#init_querry = 'SELECT posted_date,state,term,count(job_title) as count, %s from dash_table  group by posted_date, state, term '% querry
init_querry = 'SELECT * from dash_grouped_data'
skillset_sql = pd.read_sql(init_querry, con=conn)
conn.close()
'''
skillset_sql = df #pd.read_csv('grouped_april_final_for_dash.csv')
#skillset_sql['posted_date'] = skillset_sql['posted_date'].apply(lambda x:dt.strptime(x,'%Y-%m-%d'))
def manipulate_skillset(All_skills):
    # data manipulation for bar chart
    all_skills = All_skills.iloc[:,2:].sum(axis=0).sort_values(ascending= False).to_frame()
    all_skills.reset_index(inplace = True)
    all_skills.rename(columns={"index": "Skills", 0: "Counted Resumes"},inplace = True)
    all_skills.set_index('Skills', inplace =True)
    return all_skills



app = dash.Dash(__name__)
server = app.server
app.layout = html.Div(className="container scalable", children=[

    html.Div(
        id="banner",
        className="banner",
        children=[
            html.H2("US Resumes in Job Market Dashboard   V.1.1"),
            html.Img(src=app.get_asset_url("dsi.jpg")),
        ]),

    html.Div(
        id="dropdown-select-outer",
        children=[
            html.Div(
                [
                    html.P("Job Title"),
                    dcc.Dropdown(
                        id="dropdown-select",
                        options=[
                            {"label": "Advocate", "value": "Advocate"},
                            {"label": "Arts", "value": "Arts"},
                            {"label": "Automation Testing", "value": "Automation Testing"},
                            {"label": "Blockchain", "value": "Blockchain"},
                            {"label": "Business Analyst", "value": "Business Analyst"},
                            {"label": "Civil Engineer", "value": "Civil Engineer"},
                            {"label": "Data Science", "value": "Data Science"},
                            {"label": "Database", "value": "Database"},
                            {"label": "DevOps Engineer", "value": "DevOps Engineer"},
                            {"label": "DotNet Developer", "value": "DotNet Developer"},
                            {"label": "ETL Developer", "value": "ETL Developer"},
                            {"label": "Electrical Engineer", "value": "Electrical Engineer"},
                            {"label": "HR", "value": "HR"},
                            {"label": "Hadoop", "value": "Hadoop"},
                            {"label": "Health and fitness", "value": "Health and fitness"},
                            {"label": "Java Developer", "value": "Java Developer"},
                            {"label": "Mechanical Engineer", "value": "Mechanical Engineer"},
                            {"label": "Network Security Engineer", "value": "Network Security Engineer"},
                            {"label": "Operations Manager", "value": "Operations Manager"},
                            {"label": "PMO", "value": "PMO"},
                            {"label": "Python Developer", "value": "Python Developer"},
                            {"label": "SAP Developer", "value": "SAP Developer"},
                            {"label": "Sales", "value": "Sales"},
                            {"label": "Testing", "value": "Testing"},
                            {"label": "Web Designing", "value": "Web Designing"},
                            {"label": "All", "value": "all"},
                        ],
                        value="Data Science", #
                    ),
                ],
                className="selector",
            ),
            html.Div(
                [
                    html.P("Select Date Range"),
                    dcc.DatePickerRange(
                        id="date-picker-range",
                        min_date_allowed=dt(2020, 4, 20),
                        max_date_allowed=dt(
                            2021, 4, 10
                        ),  # set maximum limit according to local casting
                        initial_visible_month=dt(2020, 4, 20),
                        minimum_nights=3,
                        display_format="MMM Do, YY",
                        start_date=dt(2020, 4, 20),
                        end_date=dt(2020, 10, 30),
                    ),
                ],
                id="date-picker-outer",
                className="selector",
            ),
            html.Div(
                [
                    html.Button(
                        "reset states",
                        id="reset-button",
                        n_clicks=0,
                        n_clicks_timestamp=0,
                        style={'background-color': '#f07400'}
                    )
                ],
                title="Click to reset  graphs back to default (all states).",
                className="selector_button",
            )

        ]

    ),

    html.Div(id='output_container', children=[]),
    html.Br(),
    html.Div(className='eight columns', children=[
        html.H3(id='selected_state', style={'text-align': 'center'})]
             ),
    html.Div(id='map-container', className="eight columns",
             children=dcc.Loading(dcc.Graph(id='my_US_map', figure={}), type='circle')),
    html.Div(id='map-container2', className="four columns", children=dcc.Loading(

        children=dcc.Graph(id='skill-plot', figure={}))),
    html.Br(),
    html.Div(id='container', className="twelve columns",
             children=dcc.Loading(dcc.Graph(id='count_plot', figure={}), type='circle')
             ),
    html.Div(
        id="footer",
        className="footer",
        children=[
            html.H4("Last update on: April 2021"),
            html.H4("University of Virginia"),
            html.H6("Developed by: M. Heidarysafa"),
            html.H6("Modified by: K. Wolfe and T. Gill"),
        ]),

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(

    Output(component_id='my_US_map', component_property='figure'),
    [Input(component_id='dropdown-select', component_property='value'),
     Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date"),
     ]
)
def update_graph(option_slctd, start_d, end_d):
    print(option_slctd)
    print(type(option_slctd))
    time.sleep(1)
    # filter data in the date picker range
    # !!! overide the main job_skills with this to reduce repeteation when calling sql
    skillset_sql_map = skillset_sql[(skillset_sql['posted_date'] >= pd.Timestamp(start_d).to_pydatetime()) & (
                skillset_sql['posted_date'] < pd.Timestamp(end_d).to_pydatetime())]

    # map_data_df = skillset_sql_map.iloc[:,0:3]
    # map_data_df = map_data.to_frame()
    # map_data_df.reset_index(inplace = True)
    # map_data_df['state_name'] = map_data_df['state'].map(us_state_abbrev)
    # map_data_df.rename(columns ={'description':'count'}, inplace =True)
    if option_slctd != 'all':
        skillset_sql_map = skillset_sql_map[(skillset_sql_map["term"] == option_slctd)]
        skillset_sql_map.groupby(['posted_date', 'state'])['count'].sum().reset_index()
        skillset_sql_map.drop('posted_date', axis=1, inplace=True)
        map_data = skillset_sql_map.groupby(['state'])['count'].sum()
    else:
        skillset_sql_map.groupby(['posted_date', 'state'])['count'].sum().reset_index()
        skillset_sql_map.drop('posted_date', axis=1, inplace=True)
        map_data = skillset_sql_map.groupby(['state'])['count'].sum()

    map_data_df = map_data.to_frame()
    map_data_df.reset_index(inplace=True)
    map_data_df['state_name'] = map_data_df['state'].map(us_state_abbrev)
    # map_data_df.rename(columns ={'description':'count'}, inplace =True)

    dff = map_data_df

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state',  # careful changing this will cause not showing states
        scope="usa",
        color='count',
        hover_data=['state_name', 'count'],
        color_continuous_scale=px.colors.sequential.Bluyl,
        labels={'number of jobs: description'}
        # template='plotly_dark'
    )
    return fig


# this will reflect the effect of clicking on the us map by selecting the state
@app.callback(
    Output(component_id='selected_state', component_property='children'),
    [Input(component_id='my_US_map', component_property='clickData'),
     Input("reset-button", "n_clicks_timestamp")]
)
def get_state(clickData, reset_click):
    state = 'No State'
    now = time.time() * 1000
    if clickData is not None:
        state = clickData["points"][0]['location']
    if int(now) - int(reset_click) < 3000 and int(reset_click) > 0:
        state = "US"
    return 'You have selected: {}'.format(state)


# update the skill plot after changes on map, field
@app.callback(
    Output(component_id='skill-plot', component_property='figure'),
    [Input(component_id='my_US_map', component_property='clickData'),
     Input(component_id='dropdown-select', component_property='value'),
     Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date"),
     Input("reset-button", "n_clicks_timestamp")
     ]
)
def plot_top_skills(clickData, value, start_d, end_d, reset_click):
    time.sleep(1)
    now = time.time() * 1000
    skillset_sql_skill = skillset_sql[(skillset_sql['posted_date'] >= pd.Timestamp(start_d).to_pydatetime()) & (
                skillset_sql['posted_date'] < pd.Timestamp(end_d).to_pydatetime())]

    ####prepare skills for bar chart
    skillset = skillset_sql_skill.drop(['count', 'posted_date'], axis=1)
    if value == 'all':

        skillset = skillset
    else:
        skillset = skillset[skillset['term'] == value]
    if int(now) - int(reset_click) < 3000 and int(reset_click) > 0:

        skill = manipulate_skillset(skillset)
        fig = px.bar(skill[:20].iloc[::-1], x="Counted Resumes")
        return fig
    elif clickData and (clickData["points"][0]["location"] in list_of_us_codes):
        skillset = skillset_sql_skill.drop(['count', 'posted_date'], axis=1)
        state = clickData["points"][0]["location"]
        if value == 'all':
            state_skillset = skillset[skillset['state'] == state]
        else:

            state_skillset = skillset[(skillset['term'] == value) & (skillset['state'] == state)]
        skill = manipulate_skillset(state_skillset)
        fig = px.bar(skill[:20].iloc[::-1], x="Counted Resumes")
        return fig


    else:

        skill = manipulate_skillset(skillset)
        fig = px.bar(skill[:20].iloc[::-1], x="Counted Resumes")

        return fig


# creating count presentation plot

@app.callback(
    Output(component_id='count_plot', component_property='figure'),
    [Input(component_id='my_US_map', component_property='clickData'),
     Input(component_id='dropdown-select', component_property='value'),
     Input("date-picker-range", "start_date"),
     Input("date-picker-range", "end_date"),
     Input("reset-button", "n_clicks_timestamp")
     ]
)
def update_count_plot(clickData, value, start_d, end_d, reset_click):
    time.sleep(1)
    w_start_d = modify_start_end_date(pd.Timestamp(start_d).to_pydatetime(), forward=True)

    w_end_d = modify_start_end_date(pd.Timestamp(end_d).to_pydatetime(), forward=False)

    skillset_sql_count = skillset_sql[(skillset_sql['posted_date'] >= pd.Timestamp(w_start_d).to_pydatetime()) & (
                skillset_sql['posted_date'] < pd.Timestamp(w_end_d).to_pydatetime())]

    now = time.time() * 1000
    # count_data = skillset_sql_count.groupby(['posted_date','state','term'])['count'].sum().reset_index()
    count_week = skillset_sql_count.groupby([pd.Grouper(key='posted_date', freq='1w'), 'state', 'term'])['count'] \
        .sum().reset_index()
    count_week.rename(columns={'posted_date': 'week'}, inplace=True)
    count_week['week'] = count_week['week'].apply(lambda x: x.date())
    count_week.set_index('week', inplace=True)
    if int(now) - int(reset_click) < 3000 and int(reset_click) > 0:
        if value == 'all':
            final_plot_data = count_week.groupby('week')['count'].sum()
        else:
            final_plot_data = count_week[(count_week['term'] == value)].groupby('week')['count'].sum()
    elif clickData:
        state = clickData["points"][0]["location"]
        if value == 'all':
            final_plot_data = count_week[count_week['state'] == state].groupby('week')['count'].sum()
        else:
            final_plot_data = count_week[(count_week['state'] == state) & (count_week['term'] == value)]['count']

    else:
        if value == 'all':
            final_plot_data = count_week.groupby('week')['count'].sum()
        else:
            final_plot_data = count_week[(count_week['term'] == value)].groupby('week')['count'].sum()
    fig = px.bar(final_plot_data)
    return fig


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(port =4070, debug =True)

<IPython.core.display.Javascript object>

## Resume Classification and Screening

## Skill Extraction

### Libraries

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import spacy
from spacy.lang.en import English
from sklearn.metrics.pairwise import cosine_similarity

resumesc = pd.read_csv(resumesurl)
lexiconc = pd.read_csv(lexiconurl)

### Remove Punctuation

In [5]:
import string
punct = string.punctuation
def remove_punctuation(text):
    no_punct=[words for words in text if words not in punct]
    resume_wo_punct=''.join(no_punct)
    return resume_wo_punct
resumesc['resume_wo_punct']=resumesc['Resume'].apply(lambda x: remove_punctuation(x))

### Tokenization


In [6]:
def tokenize(text):
    split=re.split("\W+",text)
    return split
resumesc['resume_wo_punct_split']=resumesc['resume_wo_punct'].apply(lambda x: tokenize(x.lower()))

In [7]:
import nltk
from nltk.corpus import stopwords
import calendar

nltk.download('stopwords')
stopwords = stopwords.words('english')
stopwords.append('â')

years =  (['%i' %i for i in range(1900, 2100)])
months = list(calendar.month_name)
months = list(map(str.lower,months))

def remove_stopwords(text):
    text=[word for word in text if word not in stopwords]
    text=[word for word in text if word not in years]
    text=[word for word in text if word not in months]
    return text
resumesc['resume_wo_punct_split_wo_stopwords'] = resumesc['resume_wo_punct_split'].apply(lambda x: remove_stopwords(x))

resumesc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Category,Resume,resume_wo_punct,resume_wo_punct_split,resume_wo_punct_split_wo_stopwords
0,Data Science,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas nu...,"[skills, programming, languages, python, panda...","[skills, programming, languages, python, panda..."
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details \r\nMay 2013 to May 2017 BE ...,"[education, details, may, 2013, to, may, 2017,...","[education, details, uitrgpv, data, scientist,..."
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...,"[areas, of, interest, deep, learning, control,...","[areas, interest, deep, learning, control, sys..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,"[skills, â, r, â, python, â, sap, hana, â, tab...","[skills, r, python, sap, hana, tableau, sap, h..."
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details \r\n MCA YMCAUST Faridaba...,"[education, details, mca, ymcaust, faridabad, ...","[education, details, mca, ymcaust, faridabad, ..."
...,...,...,...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...,Computer Skills â¢ Proficient in MS office Wo...,"[computer, skills, â, proficient, in, ms, offi...","[computer, skills, proficient, ms, office, wor..."
958,Testing,â Willingness to accept the challenges. â ...,â Willingness to accept the challenges â P...,"[â, willingness, to, accept, the, challenges, ...","[willingness, accept, challenges, positive, th..."
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...",PERSONAL SKILLS â¢ Quick learner â¢ Eagernes...,"[personal, skills, â, quick, learner, â, eager...","[personal, skills, quick, learner, eagerness, ..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...,COMPUTER SKILLS SOFTWARE KNOWLEDGE MSPower Po...,"[computer, skills, software, knowledge, mspowe...","[computer, skills, software, knowledge, mspowe..."


## Multiclass Classification of Resumes

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

resumesmc = resumesc.copy()

jobtitles = resumesmc['Category']
for title in jobtitles:
  resumesmc['Category'] = LabelEncoder().fit_transform(resumesmc['Category'])

y = resumesmc['Category'].values

resumesmc['resume_wo_punct_split_wo_stopwords_str'] = [' '.join(map(str, l)) for l in resumesmc['resume_wo_punct_split_wo_stopwords']]
resumesmc = resumesmc['resume_wo_punct_split_wo_stopwords_str'].values
tfidf = TfidfVectorizer()
tfidf.fit(resumesmc)

X = tfidf.transform(resumesmc)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y) #80-20 split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) #60-20-20 split

In [9]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 12) #n^1/2
knn.fit(X_train,y_train)
train_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)
val_score = knn.score(X_val, y_val)
print(f' Accuracy of Training Data using KNN = {round(train_score*100,2)}%')
print(f' Accuracy of Testing Data using KNN = {round(test_score*100,2)}%')
print(f' Accuracy of Validation Data using KNN = {round(val_score*100,2)}%')

 Accuracy of Training Data using KNN = 95.14%
 Accuracy of Testing Data using KNN = 94.3%
 Accuracy of Validation Data using KNN = 91.71%


## Cosine Similarity

In [10]:
def cs(text):
  cs_df = pd.DataFrame(columns = ['Job Title', 'Similarity'])
  for category, cleaned in text.iterrows():
    skillspersonal = ','.join(cleaned[4])
    skillscommon = list(skillsdict[cleaned[0]]) #skillsdict obtained in chunk 2
    skillscommon = ','.join(skillscommon)
    comparison = [skillspersonal,skillscommon] #Creates list of 2 strings: resume and skills
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(comparison)
    matchpercentage = cosine_similarity(count_matrix)[0][1]
    matchpercentage = f'{round(matchpercentage*100,2)}%'
    cs_df.loc[len(cs_df.index)] = [cleaned[0], matchpercentage]
  return cs_df

resumescs = resumesc.copy()
results = cs(resumescs)
results

,Job Title,Similarity
0,Data Science,39.11%
1,Data Science,11.84%
2,Data Science,19.12%
3,Data Science,31.28%
4,Data Science,16.98%
...,...,...
957,Testing,20.15%
958,Testing,32.42%
959,Testing,37.8%
960,Testing,8.97%


## Skills Needed

In [11]:
sn_df = pd.DataFrame(columns = ['Job Title', 'Number of Skills Missing', 'Skills Missing'])
for index, cleaned in resumesc.iterrows():
  cleanedskillwords = []
  skills = lexicon.loc[lexicon['Category'] == cleaned[0]]
  skillslist = skills['Description'].tolist()
  skillslist = list(map(str.lower,skillslist))
  skillwords = []
  for word in skillslist:
    wordssplit = word.split()
    for word in wordssplit:
      skillwords.append(word)
  for word in skillwords:
    cleanedskillwords.append(''.join(letter for letter in word if letter.isalnum()))
  numskills = len(cleanedskillwords)
  for skill in cleaned[4]:
    if skill in cleanedskillwords:
      cleanedskillwords.remove(skill)
  #numskillsmissing = numskills and len(cleanedskillwords)/numskills or 0 #to get percentages of skills included
  numskillsmissing = f'{len(cleanedskillwords)}/{numskills}'
  skillsmissing = ', '.join(cleanedskillwords)
  sn_df.loc[len(sn_df.index)] = [cleaned[0], numskillsmissing, skillsmissing]

sn_df

,Job Title,Number of Skills Missing,Skills Missing
0,Data Science,18/45,"r, programming, learning, language, nlp, model..."
1,Data Science,39/45,"r, programming, language, science, programming..."
2,Data Science,31/45,"r, language, language, natural, language, proc..."
3,Data Science,21/45,"language, language, natural, language, modelin..."
4,Data Science,39/45,"machine, learning, r, programming, language, p..."
...,...,...,...
957,Testing,40/46,"software, manual, regression, cases, planning,..."
958,Testing,33/46,"software, manual, regression, cases, planning,..."
959,Testing,29/46,"software, manual, regression, cases, planning,..."
960,Testing,41/46,"testing, manual, testing, regression, testing,..."
